# Technical Analysis

## Integrantes:
- Efren Rivera Hernandez
- José Alfonso Martinez Ramirez
- Carola Navarro Badía
- Jose Pablo Vazquez Z

In [2]:
# Importamos las librerías necesarias
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.integrate import quad
from scipy.stats import norm
import matplotlib.pyplot as plt
from scipy.stats import lognorm

In [3]:
training = pd.read_csv("aapl_1m_train.csv")

In [4]:
training.head(5)

,Unnamed: 0,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume
0,0,1672736400,0,2023-01-03 09:00:00,130.28,131.00,130.28,131.00,8174
1,1,1672736460,0,2023-01-03 09:01:00,130.87,131.17,130.87,131.10,8820
2,2,1672736520,0,2023-01-03 09:02:00,131.18,131.24,131.17,131.17,2112
3,3,1672736580,0,2023-01-03 09:03:00,131.19,131.29,131.19,131.28,3888
4,4,1672736640,0,2023-01-03 09:04:00,131.28,131.46,131.28,131.46,5984


In [5]:
test = pd.read_csv("/Users/carolanavarro/Documents/ITESO/8vo semestre/Trading/data/aapl_1m_test.csv")

In [6]:
test.head(5)

,Unnamed: 0,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume
0,0,1693555200,0,2023-09-01 08:00:00,188.33,188.54,188.15,188.32,204
1,1,1693555260,0,2023-09-01 08:01:00,188.24,188.47,188.24,188.26,38
2,2,1693555320,0,2023-09-01 08:02:00,188.33,188.36,188.30,188.32,26
3,3,1693555380,0,2023-09-01 08:03:00,188.32,188.34,188.30,188.30,50
4,4,1693555440,0,2023-09-01 08:04:00,188.34,188.35,188.16,188.16,863


In [7]:
class Operation:
    def __init__(self, operation_type, bought_at, timestamp, 
                 n_shares, stop_loss, take_profit):
        self.operation_type = operation_type
        self.bought_at = bought_at
        self.timestamp = timestamp
        self.n_shares = n_shares
        self.sold_at = None
        self.stop_loss = stop_loss
        self.take_profit = take_profit